In [1]:
import os
import random
import obonet
import networkx
from collections import defaultdict

In [2]:
add_inverse_edge = True

reserved_edges = ["is_a"]
inverse_edges = {
    "is_a": "is_a_inverse",
}
e2i = {"is_a": 0, "is_a_inverse": 1}
i2e = {0: "is_a", 1: "is_a_inverse"}

out_dir = "./processed/gograph"

### Step 1: read GO

In [3]:
#! Some Go terms have alt_ids
obo_basic_fp = "./raw/go/go-basic.obo"
go_graph = obonet.read_obo(obo_basic_fp)

### Step 2: Pre-Process GO network

In [4]:
# Delete edges including "regulates" "part-of"
print(f"{go_graph.number_of_nodes()} nodes, {go_graph.number_of_edges()} edges exists in the GO graph.")
droped_edge = []
for u, v, e in go_graph.edges(keys=True):
    if e not in reserved_edges:
        droped_edge.append((u, v, e))
for u, v, e in droped_edge:
    go_graph.remove_edge(u, v, key=e)
print(f"{go_graph.number_of_edges()} edges are reserved in the GO graph.")


43558 nodes, 85713 edges exists in the GO graph.
70058 edges are reserved in the GO graph.


In [5]:
"""Get coarse grained of all nodes"""
coarse_grained_nodes = {}

def dp_cgnodes(node, graph):
    if node in coarse_grained_nodes:
        return
    cg_nodes = set()
    for u, v, e in graph.out_edges(node, keys=True):
        assert e in reserved_edges
        cg_nodes.add(v)
        dp_cgnodes(v, graph)
        cg_nodes = cg_nodes | coarse_grained_nodes[v]
    coarse_grained_nodes[node] = cg_nodes

for node in go_graph.nodes():
    dp_cgnodes(node, go_graph)

In [6]:
"""Add inverse edges"""
def add_inverse_edges(graph):
    additional_edges = []
    for u, v, e in graph.edges(keys=True):
        if "inverse" not in e and not graph.has_edge(v, u, key=inverse_edges[e]):
            additional_edges.append((v, u, inverse_edges[e]))

    for u, v, e in additional_edges:
        graph.add_edge(u, v, key=e)

if add_inverse_edge:
    add_inverse_edges(go_graph)

print(f"GO graph: {go_graph.number_of_nodes()} nodes, {go_graph.number_of_edges()} edges")

GO graph: 43558 nodes, 140116 edges


In [7]:
"""Separate the GO graph into three sub graphs"""
bp_component = []
mf_component = []
cc_component = []

for node, key in go_graph.nodes(data=True):
    if key['namespace'] == 'biological_process':
        bp_component.append(node)
    elif key['namespace'] == 'molecular_function':
        mf_component.append(node)
    elif key['namespace'] == 'cellular_component':
        cc_component.append(node)
    else:
        raise ValueError(f'Namespace "{key["namespace"]}" is invalid of node {node}')

# to unfreeze graph
bp_graph = networkx.MultiDiGraph(go_graph.subgraph(bp_component))
mf_graph = networkx.MultiDiGraph(go_graph.subgraph(mf_component))
cc_graph = networkx.MultiDiGraph(go_graph.subgraph(cc_component))

# print(f"GO graph: {go_graph.number_of_nodes()} nodes, {go_graph.number_of_edges()} edges")
print(f"BP graph: {bp_graph.number_of_nodes()} nodes, {bp_graph.number_of_edges()} edges")
print(f"MF graph: {mf_graph.number_of_nodes()} nodes, {mf_graph.number_of_edges()} edges")
print(f"CC graph: {cc_graph.number_of_nodes()} nodes, {cc_graph.number_of_edges()} edges")


BP graph: 28140 nodes, 102828 edges
MF graph: 11238 nodes, 27516 edges
CC graph: 4180 nodes, 9772 edges


In [8]:
"""Assign digital id to all nodes"""
def bfs(graph):
    # find root node
    root = list(graph.nodes())[0]
    while True:
        out_edges = list(graph.out_edges(root, keys=True))
        out_edges = [e for e in out_edges if "inverse" not in e[2]]
        if len(out_edges) == 0:
            break
        root = out_edges[0][1]

    idx = 0
    queue = [root]
    graph.nodes[root]['depth'] = 0

    while len(queue) != 0:
        cur_node = queue[0]
        graph.nodes[cur_node]["index"] = idx

        queue = queue[1:]
        idx += 1
        assert idx <= graph.number_of_nodes()

        in_edges = list(graph.in_edges(cur_node, keys=True))
        in_edges = [e for e in in_edges if "inverse" not in e[2]]

        for u, v, k in in_edges:
            if u in queue or graph.nodes[u].get("index", None) is not None:
                continue
            queue.append(u)
            graph.nodes[u]['depth'] = graph.nodes[v]['depth'] + 1

bfs(bp_graph)
bfs(mf_graph)
bfs(cc_graph)

In [70]:
"""Get index of all nodes"""

def get_node_index(graph):
    node_to_idx = {n: k["index"] for n, k in graph.nodes(data=True)}  
    idx_to_node = {k["index"]: n for n, k in graph.nodes(data=True)}  
    idx_to_dep = {k['index']: k['depth'] for n, k in graph.nodes(data=True)}
    return node_to_idx, idx_to_node, idx_to_dep

bp_n2i, bp_i2n, bp_i2d = get_node_index(bp_graph)
mf_n2i, mf_i2n, mf_i2d = get_node_index(mf_graph)
cc_n2i, cc_i2n, cc_i2d = get_node_index(cc_graph)

In [71]:
"""Turn ancestor into index"""

def get_ancestors(nodes, n2i):
    ancestors = {}
    for n in nodes:
        ancestors[n2i[n]] = sorted([n2i[n] for n in coarse_grained_nodes[n]])
    return ancestors

bp_ancestors = get_ancestors(bp_graph.nodes(), bp_n2i)
mf_ancestors = get_ancestors(mf_graph.nodes(), mf_n2i)
cc_ancestors = get_ancestors(cc_graph.nodes(), cc_n2i)

In [72]:
"""Get the adjacent edges of four graphs"""
def get_adj(graph, n2i):
    adj = []
    for u, v, e in graph.edges(keys=True):
        adj.append((n2i[u], e2i[e], n2i[v]))
    return adj

bp_adj = get_adj(bp_graph, bp_n2i)
mf_adj = get_adj(mf_graph, mf_n2i)
cc_adj = get_adj(cc_graph, cc_n2i)
# go_adj = get_adj(go_graph, go_n2i)

In [73]:
"""Get descriptions of all nodes"""
import re
p = re.compile('(\[.+\])|\"')

def get_desc(graph, n2i):
    node_descs = {}
    for node, data in graph.nodes(data=True):
        desc = re.sub(p, "", data['def']).strip()
        node_descs[n2i[node]] = desc
    return node_descs

bp_desc = get_desc(bp_graph, bp_n2i)
mf_desc = get_desc(mf_graph, mf_n2i)
cc_desc = get_desc(cc_graph, cc_n2i)

In [74]:
"""Get leaf terms"""
def get_leaf(graph, n2i):
    leaves = []
    for node in graph.nodes():
        out_edges = list(graph.out_edges(node, keys=True))
        out_edges = [e for e in out_edges if "inverse" in e[2]]
        if len(out_edges) == 0:
            leaves.append((node, n2i[node]))
    return sorted(leaves, key=lambda x: x[1])

bp_leaves = get_leaf(bp_graph, bp_n2i)
mf_leaves = get_leaf(mf_graph, mf_n2i)
cc_leaves = get_leaf(cc_graph, cc_n2i)

In [75]:
# """Get alts id of GO Terms"""
# def get_alts(nodes, graph):
#     for idx, node in enumerate(nodes):
#         goid = node[2]
#         alt_ids = graph.nodes[goid].get("alt_id", [])
#         node.append(" ".join(alt_ids))

# get_alts(bp_nodes, bp_graph)
# get_alts(mf_nodes, mf_graph)
# get_alts(cc_nodes, cc_graph)

### Step 3: Store Go graph

In [76]:
# networkx.write_gpickle(go_graph, os.path.join(out_dir, "go.nx"))

def write_file(fp, lines, header=None):
    with open(fp, "w") as f:
        if header is not None:
            f.write(header)
        f.writelines(lines)


if add_inverse_edge:
    # store e2i
    with open(os.path.join(out_dir, "edge_index.tsv"), "w") as f:
        f.write("Edge\tIndex\n")
        f.writelines([f"{e}\t{i}\n" for e, i in e2i.items()])


def store(_dir, graph, n2i, i2n, i2d, leaves, ances, desc, adj):
    os.makedirs(_dir, exist_ok=True)

    networkx.write_gpickle(graph, os.path.join(_dir, "graph.nx"))

    # store node2idx 
    write_file(
        os.path.join(_dir, "term_index.tsv"),
        [f"{n}\t{i}\n" for n, i in sorted([(n, i) for n, i in n2i.items()], key=lambda x: x[1])],
        header="Term\tIndex\n")

    # store leaves
    write_file(
        os.path.join(_dir, 'term_leaf.tsv'),
        [f"{n}\t{i}\t{i2d[i]}\n" for n, i in leaves], header="Leaf\tIndex\tDepth\n")

    # sotre term depth
    write_file(
        os.path.join(_dir, "term_depth.tsv"),
        [f"{i2n[i]}\t{i}\t{i2d[i]}\n" for i in range(len(i2d))],
        header="Term\tIndex\tDepth\n")

    # store father nodes
    write_file(
        os.path.join(_dir, "term_ances.tsv"),
        [f"{i2n[i]}\t{i}\t{' '.join(list(map(str, ances[i])))}\n" for i in range(len(ances))],
        header="Term\tIndex\tAncestors\n")

    # store node descriptions
    write_file(
        os.path.join(_dir, "term_desc.tsv"),
        [f"{i2n[i]}\t{i}\t{desc[i]}\n" for i in range(len(desc))],
        header="Term\tIndex\tName\n")

    # store adjacant matrix
    write_file(
        os.path.join(_dir, "adj.tsv"),
        [f"{i2n[si]}\t{i2e[ei]}\t{i2n[ti]}\t{si}\t{ei}\t{ti}\n" for si, ei, ti in sorted(adj, key=lambda x: x[0])],
        header="Souce\tEdge\tTarget\tSIndex\tEIndex\tTIndex\n")

store(os.path.join(out_dir, "bp"), bp_graph, bp_n2i, bp_i2n, bp_i2d, bp_leaves, bp_ancestors, bp_desc, bp_adj)
store(os.path.join(out_dir, "mf"), mf_graph, mf_n2i, mf_i2n, mf_i2d, mf_leaves, mf_ancestors, mf_desc, mf_adj)
store(os.path.join(out_dir, "cc"), cc_graph, cc_n2i, cc_i2n, cc_i2d, cc_leaves, cc_ancestors, cc_desc, cc_adj)